In [ ]:
##

#RAG Pipeline

In [1]:
!pip install -qU langchain-community faiss-cpu tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00


In [2]:
!pip install  langchain-core langchain-huggingface langgraph langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool

from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
import pathlib
from langchain_huggingface import HuggingFaceEmbeddings
import faiss
from langchain_community.vectorstores import FAISS


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [36]:
def load_vector_db_from_local(file_name, embedding):
    """
    Loads the vector DB from a local file.
    """
    return FAISS.load_local(file_name, embedding, allow_dangerous_deserialization=True)

def get_default_embeddings(model_name="sentence-transformers/all-mpnet-base-v2"):
    """Return the default embedding model."""
    return HuggingFaceEmbeddings(model_name=model_name)


def build_RAG_pipeline(llm, vector_store):
    @tool(response_format="content_and_artifact")
    def retrieve(query: str):
        """Retrieve relevant information from the University of Connecticut knowledge base based on the user's query and provide context to help answer the question effectively."""
        retrieved_docs = search_vector_db(query, vector_store)
        serialized = "\n\n".join(
            (f"Source: {doc[0].metadata}\n" f"Content: {doc[0].page_content}")
            for doc in retrieved_docs
        )
        return serialized, retrieved_docs

    def query_or_respond(state: MessagesState):
        """Generate tool call for retrieval or respond."""
        llm_with_tools = llm.bind_tools([retrieve])
        system_message_content = """You are an assistant designed to provide answers based on the University of Connecticut (UConn) knowledge base.
        If a query is related to UConn and requires more context, call the tool with an appropriate query to fetch relevant documents from the knowledge base.

        - Ensure the query is specific and clearly related to the subject matter to retrieve the most relevant documents.
        - If you feel the query could be rewritten to fetch more relevant content, feel free to adjust or rephrase it to better align with the knowledge base.
         """
        prompt = [SystemMessage(system_message_content)] + state['messages']
        print(f"Input message ------{prompt}")
        response = llm_with_tools.invoke(prompt)
        return {"messages": [response]}

    def generate(state: MessagesState):
        """Generate answer."""
        # Get generated ToolMessages
        recent_tool_messages = []
        for message in reversed(state["messages"]):
            if message.type == "tool":
                recent_tool_messages.append(message)
            else:
                break
        tool_messages = recent_tool_messages[::-1]

        # Format into prompt
        docs_content = "\n\n".join(doc.content for doc in tool_messages)

        system_message_content = (
            """"You are an assistant designed to answer questions related to the University of Connecticut (UConn) knowledge base. When responding,
            utilize the provided context to generate accurate answers. If you are uncertain about the answer,
            acknowledge that you don’t know. For questions that inquire "How," provide a detailed, step-by-step explanation. Whenever necessary,
             include metadata with the URL of the original source site to ensure proper attribution."""
            "\n\n"
            f"{docs_content}"
        )
        conversation_messages = [
            message
            for message in state["messages"]
            if message.type in ("human", "system")
               or (message.type == "ai" and not message.tool_calls)
        ]
        prompt = [SystemMessage(system_message_content)] + conversation_messages

        # Run
        response = llm.invoke(prompt)
        return {"messages": [response]}

    graph_builder = StateGraph(MessagesState)
    tools = ToolNode([retrieve])
    graph_builder.add_node(query_or_respond)
    graph_builder.add_node(tools)
    graph_builder.add_node(generate)

    graph_builder.set_entry_point("query_or_respond")
    # graph_builder.add_edge("query_or_respond", "tools")
    graph_builder.add_conditional_edges(
        "query_or_respond",
        tools_condition,
        {END: END, "tools": "tools"},
    )
    graph_builder.add_edge("tools", "generate")
    graph_builder.add_edge("generate", END)
    graph = graph_builder.compile()

    return graph

In [37]:
def search_vector_db(query, vector_store, k=5, filter=None):
    """
    Perform a search in the vector database by querying for the most similar documents.

    Parameters:
    - query (str): The search query string for which similar documents are being sought.
    - vector_store (FAISS): The FAISS vector store that contains the document vectors to search against.
    - k (int, optional): The number of nearest neighbors (documents) to return. Default is 5.
    - filter (dict, optional): A filter to apply to the search (e.g., metadata filtering). Default is None.

    Returns:
    - list: A list of the top `k` most similar documents, along with their similarity scores.
    """
    # Perform the similarity search in the vector store using the provided query and filter (if any)
    documents = vector_store.similarity_search_with_score(query, k=k, filter=filter)

    return documents

In [38]:
data_dir = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/Capstone")
vector_db_location = data_dir / "faiss_index"
embedding = get_default_embeddings()

# Load the vector store from disk
vector_store = load_vector_db_from_local(vector_db_location, embedding)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [39]:
graph = build_RAG_pipeline(llm, vector_store)

In [40]:
input_message = "Tell me more about two-factor authentication in netid"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me more about two-factor authentication in netid
Input message ------[SystemMessage(content='You are an assistant designed to provide answers based on the University of Connecticut (UConn) knowledge base.\n        If a query is related to UConn and requires more context, call the tool with an appropriate query to fetch relevant documents from the knowledge base. \n         \n        - Ensure the query is specific and clearly related to the subject matter to retrieve the most relevant documents.\n        - If you feel the query could be rewritten to fetch more relevant content, feel free to adjust or rephrase it to better align with the knowledge base.\n         ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me more about two-factor authentication in netid', additional_kwargs={}, response_metadata={}, id='964bf34b-1ab8-4d49-b650-3239b1af93cc')]
=============================